Dask

In [ ]:
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(queue='batch',cores=8, project='gfdl_f',memory='60GB', walltime='00:30:00' )

In [ ]:
cluster.close()

In [ ]:
from dask.distributed import Client

In [ ]:
client = Client(cluster)

In [ ]:
client

Netcdf time average

### SST OM4p25

In [ ]:
import cartopy
import cmocean
import matplotlib.pyplot as plt
import matplotlib.colors
import matplotlib.colorbar
import xarray as xr
import numpy

In [ ]:
OM4p25 = xr.open_mfdataset('/work/Nkeh.Boh/SST/OM4p25-100yr/Netfile/*.nc')

In [ ]:
OM4p25 = OM4p25.chunk(200, 1080, 1440)

In [ ]:
OM4p25

In [ ]:
#OM4p25 = OM4p25.chunk(1200, 1080, 1440)

In [ ]:
%%time
temp_mean = OM4p25['tos'].mean(dim='time').load()# need to do chunking here, while running data

In [ ]:
temp_mean.shape

In [ ]:
temp_mean

In [ ]:
W25 = xr.open_mfdataset('/archive/gold/datasets/OM4_025/obs/WOA05_ptemp_salt_monthly.v20141007.nc')

In [ ]:
wtemp_mean  = numpy.mean(W25['ptemp'][:,0,:,:],axis=0)

# Plots

In [ ]:
plt.pcolormesh(temp_mean)
plt.title('a) OM4p25')

In [ ]:
fig = plt.figure(figsize=(8,5))
vmin,vmax,ci,cmap = -2.25,2.25,.5,plt.cm.RdYlBu_r
axes = []
cilev = numpy.arange(vmin-ci,vmax+ci*2,ci)
norm = matplotlib.colors.BoundaryNorm(boundaries=cilev, ncolors=cmap.N)

q = temp_mean  #

ax = plt.axes(projection=cartopy.crs.Robinson(central_longitude=-155))

im = ax.pcolormesh(q, vmin=vmin, vmax=vmax)
#stats(ax, a25, q, 'OMp25')
ax.coastlines()
axes.append(ax)
plt.title('a) OM4p25')

In [ ]:
G25 = netCDF4.Dataset('/archive/gold/datasets/OM4_025/mosaic.v20170622.unpacked/ocean_hgrid.nc')
G5 = netCDF4.Dataset('/archive/gold/datasets/OM4_05/mosaic_ocean.v20180227.unpacked/ocean_hgrid.nc')
M25 = netCDF4.Dataset('/archive/gold/datasets/OM4_025/mosaic.v20170622.unpacked/ocean_mask.nc')
M5 = netCDF4.Dataset('/archive/gold/datasets/OM4_05/mosaic_ocean.v20180227.unpacked/ocean_mask.nc')

OM4p25_av, W25_av = temp_mean, wtemp_mean

xq25 = G25.variables['x'][:][::2,::2]
yq25 = G25.variables['y'][:][::2,::2]
a25 = G25.variables['area'][:]; a25 = a25[::2,::2]+a25[1::2,1::2]+a25[1::2,::2]+a25[1::2,::2]
m25 = M25.variables['mask'][:]; a25 = a25*m25


def stats(ax, area, anomaly, label):
    mn = (anomaly*area).sum()/area.sum()
    sd = numpy.sqrt( ((anomaly-mn)**2*area).sum()/area.sum() )
    rms = numpy.sqrt( (anomaly**2*area).sum()/area.sum() )
    qmn, qmx = anomaly.min(), anomaly.max()
    print(label, 'mean =', mn, 'sd =', sd, 'rms =', rms, 'min =', qmn, 'max =', qmx )
    bb = ax.get_position()
    plt.gcf().text(bb.x0,bb.y1+.01,'mean=%.3f$^\circ$C'%mn, horizontalalignment='left')
    plt.gcf().text(bb.x1,bb.y1+.01,'rms=%.3f$^\circ$C'%rms, horizontalalignment='right')


fig = plt.figure(figsize=(8,5))
vmin,vmax,ci,cmap = -2.25,2.25,.5,plt.cm.RdYlBu_r
axes = []
cilev = numpy.arange(vmin-ci,vmax+ci*2,ci)
norm = matplotlib.colors.BoundaryNorm(boundaries=cilev, ncolors=cmap.N)

q = OM4p25_av - W25_av  #

ax = plt.axes(projection=cartopy.crs.Robinson(central_longitude=-155))

im = ax.pcolormesh(xq25, yq25, q, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, vmin=vmin, vmax=vmax)
stats(ax, a25, q, 'OMp25')
ax.coastlines()
axes.append(ax)
plt.title('a) OM4p25')